In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Input,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE
import random
from collections import deque


# define the model
model = Sequential([

    Dense(128, activation = 'relu',input_shape = (11,)),
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(3)
])

adam = Adam(0.001)
loss_f = MSE

# define the memory
memory = deque(maxlen = 100000)

# define the hyperparamters
gamma = 0.9
batch_size = 64
epsilon = 1
epsilon_min = 0.0001
epsilon_decay = 0.998

# movement
right = 1
down = 2
left = 3
up = 4

def move():
     global x1_change,y1_change,snake_block
     global action , direction
     global x1 , y1

     actions = np.argmax(action)
     clockwise = [right,down,left,up]
     idx = clockwise.index(direction)

     if actions == 0:
          new_dir = clockwise[idx]
     elif actions == 1:
          idx_new = (idx + 1) % 4
          new_dir = clockwise[idx_new]
     else:
          idx_new = (idx - 1) % 4
          new_dir = clockwise[idx_new]

     direction = new_dir

     if direction == 1:
          x1_change = snake_block
          y1_change = 0
     elif direction == 3:
          x1_change = -snake_block
          y1_change = 0
     elif direction == 4:
          y1_change = -snake_block
          x1_change = 0
     else:
          y1_change = snake_block
          x1_change = 0

     x1 += x1_change
     y1 += y1_change

def collision(x1s,y1s):
    global dis_width,dis_height,snake_List

    if  x1s >= (dis_width)  or x1s <= -20 or y1s >= (dis_height)  or y1s <= -20:
         return True
    if [x1s,y1s] in snake_List[:-1]:
         return True
    else:
         return False
    
def distance():
     global x1 , y1
     global foodx , foody
     return np.sqrt( (x1 - foodx)**2 + (y1 - foody)**2 )


def step():
     global x1,y1
     global foodx , foody
     global snake_List

     old_distance = distance()
     move()
     new_distance = distance()
     reward = 0
     if x1 > dis_width -20  or x1 < 0 or y1 > dis_height - 20  or y1 < 0 or [x1,y1] in snake_List[:-1]:
          reward = -100
          return reward , True
     
     if new_distance < old_distance:
          reward = 5
     else:
          reward = -5

     if x1 == foodx and y1 == foody:
          reward = 100


     return reward , False

def get_state():

     global direction
     global x1
     global y1
     global foodx , foody

     dir_r = (direction == right)
     dir_d = (direction == down)
     dir_l = (direction == left)
     dir_u = (direction == up)

     x1_right = x1 + 20 
     x1_left = x1 - 20
     y1_up = y1 - 20
     y1_down = y1 + 20 

     state = [
          ((dir_r and collision(x1_right,y1))or
           (dir_l and collision(x1_left,y1))or  # danger straight
           (dir_u and collision(x1,y1_up))or
           (dir_d and collision(x1,y1_down))),

          ((dir_r and collision(x1,y1_down))or
           (dir_l and collision(x1,y1_up))or  # danger right
           (dir_u and collision(x1_right,y1))or
           (dir_d and collision(x1_left,y1))),

           ((dir_r and collision(x1,y1_up))or
           (dir_l and collision(x1,y1_down))or  # danger left
           (dir_u and collision(x1_left,y1))or
           (dir_d and collision(x1_right,y1))),

           dir_r,
           dir_d,
           dir_l,
           dir_u,

           x1 > foodx,
           x1 < foodx,
           y1 > foody,
           y1 < foody
]
     return np.array(state,dtype=int)

def get_action(state , epsilon):
         if epsilon > epsilon_min:
            epsilon = epsilon * epsilon_decay
         else:
            epsilon = epsilon_min

         new_action = [0, 0, 0]

         if np.random.rand() > epsilon:
            print(1)
            action_choice = np.random.randint(0, 2)
            new_action[action_choice] = 1
         else:
            
            Q_values = model.predict(np.array([state]))
            action_choice = np.argmax(Q_values[0])
            new_action[action_choice] = 1

         return new_action
def get_experience():
        if len(memory) > batch_size:
        
            mini = random.sample(memory,batch_size)
        else:
            mini = memory

        states , actions , rewards , next_states , dones = [],[],[],[],[]

        for state, action, reward, next_state, done in mini:
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(done)

        states = np.array(states)
        next_states = np.array(next_states)
        actions = np.array(actions)
        rewards = np.array(rewards)
        dones = np.array(dones)


        return states, actions, rewards, next_states, dones
            
def training_step():
        experiences = get_experience()
        states, actions, rewards, next_states, dones = experiences
        next_Q_values = model.predict(next_states)
        max_next_Q_values = np.max(next_Q_values, axis=1)
        target_Q_values = (rewards + gamma * max_next_Q_values)

        with tf.GradientTape() as tape:
            all_Q_values = model(states)
            Q_values = tf.reduce_sum(
                all_Q_values * actions, axis=1, keepdims=True)
            loss = tf.reduce_mean(loss_f(target_Q_values, Q_values))
        grads = tape.gradient(loss, model.trainable_variables)
        adam.apply_gradients(
            zip(grads, model.trainable_variables))



def remember(state, action , reward , next_state , done):
    memory.append((state, action , reward , next_state , done))



import pygame
import time
import random
 
pygame.init()

 
white = (255, 255, 255)
yellow = (255, 255, 102)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)
 
dis_width = 640
dis_height = 480
 
dis = pygame.display.set_mode((dis_width, dis_height))
pygame.display.set_caption('Neural Network Learns to play Snake')
 
clock = pygame.time.Clock()
 
snake_block = 20
snake_speed = 100
 
font_style = pygame.font.SysFont("bahnschrift", 25)
score_font = pygame.font.SysFont("comicsansms", 35)

def our_snake(snake_block, snake_list):
    for x in snake_list:
        pygame.draw.rect(dis, white, [x[0], x[1], snake_block, snake_block])




for i in range(150):
    
 
    x1 = dis_width / 2
    y1 = dis_height / 2
 
    x1_change = 0
    y1_change = 0
 
    snake_List = []
    Length_of_snake = 2

    foodx = round(random.randrange(0, dis_width - snake_block) / 20.0) * 20.0
    foody = round(random.randrange(0, dis_height - snake_block) / 20.0) * 20.0

    x1_change = snake_block
    y1_change = 0

    direction = left

    while True:
        state = get_state()
        action = get_action(state , epsilon)
        reward , done = step()
        next_state = get_state()
        remember(state, action , reward , next_state , done)

        for event in pygame.event.get():
             if event.type == pygame.QUIT:
                  pygame.quit()
                  break


        if x1 == foodx and y1 == foody:
            foodx = round(random.randrange(0, dis_width - snake_block) / 20.0) * 20.0
            foody = round(random.randrange(0, dis_height - snake_block) / 20.0) * 20.0
            Length_of_snake += 1
            

        dis.fill(black)
        pygame.draw.rect(dis, red, [foodx, foody, snake_block, snake_block])
        snake_Head = []
        snake_Head.append(x1)
        snake_Head.append(y1)
        snake_List.append(snake_Head)



        if len(snake_List) > Length_of_snake:
            del snake_List[0]
    
        our_snake(snake_block, snake_List)
 
        pygame.display.update()
 
        clock.tick(snake_speed)

        training_step()
        
        if done:
             break
        
pygame.quit()
model.save('agent.h5')
model.save_weights('agent_weights.h5')


pygame 2.5.2 (SDL 2.28.3, Python 3.7.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


error: display Surface quit